In [3]:
# as of time of writing may 25, this takes 5 minutes to gen features. still slow :/
import numpy as np
import pandas as pd
import os, time, gc, pickle, random
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
from keras.preprocessing import text, sequence
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F
from pytorch_pretrained_bert import BertTokenizer, BertModel
from bert_embedding import BertEmbedding
import apex # used for 16 bit
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
import mxnet as mx # used for GPU
#for getting num good and bad words
from wordcloud import STOPWORDS
from collections import defaultdict
import operator
import swifter # speed up feature gen - multiple cores

# Logging for BERT
import logging
logging.basicConfig(level=logging.INFO)


from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score

Using TensorFlow backend.
/home/curtis/miniconda3/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [4]:
# helper functions
string.printable
ascii_chars = string.printable
ascii_chars += " áéíóúàèìòùâêîôûäëïöüñõç"

#checks if a string of text contains any nonenglish characters (excluding punctuations, spanish, and french characters)
def contains_non_english(text):
    if all(char in ascii_chars for char in text):
        return 0
    else:
        return 1
    
#clean non english characters from string of text
def remove_non_english(text):
    return ''.join(filter(lambda x: x in ascii_chars, text))


def get_first_word(word):
    if(type(word) != "float"):
        return word.split(" ")[0]
    return "-1"

def get_caps_vs_length(df):
    df['caps_vs_length'] = df['num_caps'].divide(df['num_chars'])
    df.loc[~np.isfinite(df['caps_vs_length']), 'caps_vs_length'] = 0
    
"""    mask = (df['num_chars'] != 0)
    df_valid = df[mask]
    
    df.loc[mask, 'caps_vs_length'] = df_valid['num_caps'] / df_valid['num_chars']"""

def get_unique_word_over_num_words(df):
    df['unique_word_over_num_words'] = df['num_unique_words'].divide(df['num_words'])
    df.loc[~np.isfinite(df['unique_word_over_num_words']), 'unique_word_over_num_words'] = 0
    
def get_avg_word_len(df):
    df['avg_word_len'] = df['total_word_length'].divide(df['num_words'])
    df.loc[~np.isfinite(df['avg_word_len']), 'avg_word_len'] = 0

def get_avg_unique_word_len(df):
    df['avg_unique_word_len'] = df['total_unique_word_length'].divide(df['num_unique_words'])
    df.loc[~np.isfinite(df['avg_unique_word_len']), 'avg_unique_word_len'] = 0
    
def calc_max_word_len(sentence):
    maxLen = 0
    for word in sentence:
        maxLen = max(maxLen, len(word))
    return maxLen

def calc_min_word_len(sentence):
    minLen = 999999
    for word in sentence:
        minLen = min(minLen, len(word))
    return minLen

def calc_total_word_len(sentence):
    cnt = 0
    for x in sentence:
        cnt+=len(x)
    return cnt

def calc_total_unique_word_len(sentence):
    words = set(sentence)
    return calc_total_word_len(words)

#removes all single characters except for "I" and "a"
def remove_singles(text):
    return ' '.join( [w for w in text.split() if ((len(w)>1) or (w.lower() == "i") or (w.lower() == "a"))] )
    
#combines multiple whitespaces into single
def clean_text(x):
    x = str(x)
    for punct in "&/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~`" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&':
        x = x.replace(punct, '')
    x = re.sub( '\s+', ' ', x).strip()
    
    
# Text cleaning
# TODO: speed up this func
def pad_chars(text,punct):
    for p in punct:
        text = re.sub('(?<=\w)([!?,])', r' \1', text)
    return text
    
symbols_iv = """?,./-()"$=…*&+′[ɾ̃]%:^\xa0\\{}–“”;!<`®ạ°#²|~√_α→>—£，。´×@π÷？ʿ€の↑∞ʻ℅в•−а年！∈∩⊆§℃θ±≤͡⁴™си≠∂³ி½△¿¼∆≥⇒¬∨∫▾Ω＾γµº♭ー̂ɔ∑εντσ日Γ∪φβ¹∘¨″⅓ɑː✅✓（）∠«»்ுλ∧∀،＝ɨʋδɒ¸☹μΔʃɸηΣ₅₆◦·ВΦ☺❤♨✌≡ʌʊா≈⁰‛：ﬁ„¾ρ⟨⟩˂⅔≅－＞¢⁸ʒは⬇♀؟¡⋅ɪ₁₂ɤ◌ʱ、▒ْ；☉＄∴✏ωɹ̅।ـ☝♏̉̄♡₄∼́̀⁶⁵¦¶ƒˆ‰©¥∅・ﾟ⊥ª†ℕ│ɡ∝♣／☁✔❓∗➡ℝ位⎛⎝¯⎞⎠↓ɐ∇⋯˚⁻ˈ₃⊂˜̸̵̶̷̴̡̲̳̱̪̗̣̖̎̿͂̓̑̐̌̾̊̕\x92"""        

def split_off_symbols_iv(x):
    for punct in symbols_iv:
        x = x.replace(punct, f' {punct} ')
    return x
    
def neutrailize_bad_words(train,test):
    train1_df = train[train["target"]==1]
    train0_df = train[train["target"]==0]

    ## custom function for ngram generation ##
    def generate_ngrams(text, n_gram=1):
        token = [token for token in text.lower().split(" ") if token != "" if token not in STOPWORDS]
        ngrams = zip(*[token[i:] for i in range(n_gram)])
        return [" ".join(ngram) for ngram in ngrams]

    freq_dict_bad = defaultdict(int)
    for sent in train1_df["comment_text"]:
        for word in generate_ngrams(sent):
            freq_dict_bad[word] += 1
    freq_dict_bad = dict(freq_dict_bad)

    freq_dict_good = defaultdict(int)
    for sent in train0_df["comment_text"]:
        for word in generate_ngrams(sent):
            freq_dict_good[word] += 1
    freq_dict_good = dict(freq_dict_good)

    bad_words = sorted(freq_dict_bad, key=freq_dict_bad.get, reverse=True)[:1000]
    good_words = sorted(freq_dict_good, key=freq_dict_good.get, reverse=True)[:1000]

    print("--- Generating num_bad_words")
    train["num_bad_words"] = train["comment_text"].map(lambda x: num_bad_words(x))
    test["num_bad_words"] = test["comment_text"].map(lambda x: num_bad_words(x))

    print("--- Generating num_good_words")
    train["num_good_words"] = train["comment_text"].map(lambda x: num_good_words(x))
    test["num_good_words"] = test["comment_text"].map(lambda x: num_good_words(x))

    return x

In [5]:
def preprocess(data):
    '''
    Credit goes to https://www.kaggle.com/gpreda/jigsaw-fast-compact-solution
    '''
    
    # Feature generation
    
    def gen_feats(df):
        start_time = time.time()
            
        print("--- Generating non_eng")
        df["non_eng"] = df["comment_text"].swifter.apply(lambda x: contains_non_english(x))
        print("--- Finished Gen Feats. Took %s seconds ---" % (time.time() - start_time))

        print("--- Generating first_word")
        df["first_word"] = df["comment_text"].swifter.apply(lambda x: get_first_word(x))

        print("--- Generating num_chars (num chars)")
        df['num_chars'] = df['comment_text'].swifter.apply(len)

        print("--- Generating num_caps")
        df['num_caps'] = df['comment_text'].swifter.apply(lambda comment: sum(1 for c in comment if c.isupper()))

        print("--- Generating caps_vs_length")
        get_caps_vs_length(df)

        print("--- Generating num_exclamation_marks")
        df['num_exclamation_marks'] = df['comment_text'].apply(lambda comment: comment.count('!'))

        print("--- Generating num_question_marks")
        df['num_question_marks'] = df['comment_text'].apply(lambda comment: comment.count('?'))

        print("--- Generating num_punctuation")
        df['num_punctuation'] = df['comment_text'].apply(lambda comment: sum(comment.count(w) for w in '.,;:'))

        print("--- Generating num_symbols")
        df['num_symbols'] = df['comment_text'].apply(lambda comment: sum(comment.count(w) for w in '*&$%'))

        print("--- Generating num_words")
        df['num_words'] = df['comment_text'].swifter.apply(lambda comment: len(re.sub(r'[^\w\s]','',comment).split(" ")))

        print("--- Generating num_unique_words")
        df['num_unique_words'] = df['comment_text'].swifter.apply(lambda comment: len(set(w for w in comment.split())))

        print("--- Generating unique_word_over_num_words")
        get_unique_word_over_num_words(df)

        print("--- Generating num_smilies")
        df['num_smilies'] = df['comment_text'].apply(lambda comment: sum(comment.count(w) for w in (':-)', ':)', ';-)', ';)')))

        print("--- Generating num_sentences")
        df['num_sentences'] = df['comment_text'].swifter.apply(lambda comment: len(re.split(r'[.!?]+', comment)))

        print("--- Generating max_word_len")
        df['max_word_len'] = df['comment_text'].swifter.apply(lambda comment: calc_max_word_len(re.sub(r'[^\w\s]','',comment).split(" ")))
        
        print("--- Generating min_word_len")
        df['max_word_len'] = df['comment_text'].swifter.apply(lambda comment: calc_min_word_len(re.sub(r'[^\w\s]','',comment).split(" ")))
        
        print("--- Generating total_word_length (num of chars in words)")
        df['total_word_length'] = df['comment_text'].swifter.apply(lambda comment: calc_total_word_len(re.sub(r'[^\w\s]','',comment).split(" ")))
        
        print("--- Generating avg_word_len")
        get_avg_word_len(df)
        
        print("--- Generating total_unique_word_length (num of chars in words)")
        df['total_unique_word_length'] = df['comment_text'].swifter.apply(lambda comment: calc_total_unique_word_len(re.sub(r'[^\w\s]','',comment).split(" ")))
        
        print("--- Generating avg_unique_word_len")
        get_avg_unique_word_len(df)
        
        print("--- Finished Gen Feats. Took %s seconds ---" % (time.time() - start_time))
        

    def cleanText(df):
        start_time = time.time()
        df['comment_text'] = df['comment_text'].swifter.apply(lambda x: split_off_symbols_iv(x)) #increase score
        """print("--- cleaning text")
        df["comment_text"] = df["comment_text"].apply(lambda x: clean_text(x))

        print("--- remove single characters")
        df["comment_text"] = df["comment_text"].apply(lambda x: remove_singles(x))

        print("--- cleaning numbers")
        df["comment_text"] = df["comment_text"].apply(lambda x: clean_numbers(x))

        print("--- cleaning misspellings")
        df["comment_text"] = df["comment_text"].apply(lambda x: replace_typical_misspell(x))

        print("--- filling missing values")
        #clean chinese, korean, japanese characters
        print('cleaning characters')
        df["comment_text"] = df["comment_text"].map(lambda x: remove_non_english(x))
        
        ## fill up the missing values
        df["comment_text"].fillna("").values"""
        print("--- Finished cleaning text. Took %s seconds ---" % (time.time() - start_time))

        
    gen_feats(data)
    #data["comment_text"] = data["comment_text"].astype(str).apply(lambda x: pad_chars(x, punct))
    #cleanText(data)
    # print("--- Neutralizing bad words")
    # neutrailize_bad_words(train,test)
    
    return data

In [4]:
DATA_SIZE = ""
print("--- Loading Files")
start_time = time.time()
train = pd.read_hdf('../input/train.h5')
test = pd.read_hdf('../input/test.h5')
print("--- Finished Loading %s" % (time.time() - start_time))

if DATA_SIZE == "small":
    print("Using small data")
    train = train[:100]
    test = test[:100]
elif DATA_SIZE == "med":
    print("Using med data")
    train = train[:10000]
    test = test[:10000]

print("--- Preprocess")
start_time = time.time()
preprocess(train)
preprocess(test)
print("--- Finished preprocess %s" % (time.time() - start_time))

--- Loading Files
--- Finished Loading 2.3180928230285645
--- Preprocess
--- Generating non_eng



--- Finished Gen Feats. Took 25.815589904785156 seconds ---
--- Generating first_word



--- Generating num_chars (num chars)



--- Generating num_caps



--- Generating caps_vs_length
--- Generating num_exclamation_marks
--- Generating num_question_marks
--- Generating num_punctuation
--- Generating num_symbols
--- Generating num_words


...


--- Generating max_word_len



--- Generating min_word_len



--- Generating total_word_length (num of chars in words)



--- Generating avg_word_len
--- Generating total_unique_word_length (num of chars in words)



--- Generating avg_unique_word_len
--- Finished Gen Feats. Took 10.602014064788818 seconds ---
--- Finished preprocess 200.95978951454163


In [5]:
split_perc = 0.9

train_len = len(train)
split_point = int(split_perc*train_len)

#first_word omitted
features = ['non_eng','num_chars','num_caps','caps_vs_length','num_exclamation_marks','num_question_marks','num_punctuation','num_symbols','num_words','num_unique_words','unique_word_over_num_words','num_smilies','num_sentences','max_word_len','max_word_len','total_word_length','avg_word_len','total_unique_word_length','avg_unique_word_len']
#x_train = train[:split_point][features]
#x_test = test[split_point:][features]

#y_train = train[:split_point]['target']
#y_aux_train = [[] for x in range(len(x_train))]

x_train = train[features]
y_train = np.where(train['target'] >= 0.5, 1, 0)

x_test = test[features]

In [6]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

In [7]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out


In [8]:
class SequenceBucketCollator():
    def __init__(self, choose_length, sequence_index, length_index, label_index=None):
        self.choose_length = choose_length
        self.sequence_index = sequence_index
        self.length_index = length_index
        self.label_index = label_index
        
    def __call__(self, batch):
        batch = [torch.stack(x) for x in list(zip(*batch))]
        
        sequences = batch[self.sequence_index]
        lengths = batch[self.length_index]
        
        length = self.choose_length(lengths)
        mask = torch.arange(start=maxlen, end=0, step=-1) < length
        padded_sequences = sequences[:, mask]
        
        batch[self.sequence_index] = padded_sequences
        
        if self.label_index is not None:
            return [x for i, x in enumerate(batch) if i != self.label_index], batch[self.label_index]
    
        return batch

In [9]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def train_model(model, train, test, loss_fn, output_dim, lr=0.001,
                batch_size=512, n_epochs=4, n_epochs_embed=2,
                enable_checkpoint_ensemble=True):
    
    
    param_lrs = [{'params': param, 'lr': lr} for param in model.parameters()]
    optimizer = torch.optim.Adam(param_lrs, lr=lr)

    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda epoch: 0.6 ** epoch)
    
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
    all_test_preds = []
    checkpoint_weights = [2 ** epoch for epoch in range(n_epochs)]
    
    for epoch in range(n_epochs):
        start_time = time.time()
        
        scheduler.step()
        
        model.train() #set model to train mode
        avg_loss = 0.
        
        for data in tqdm(train_loader, disable=False):
            
            #training loop
            x_batch = data[:-1]
            y_batch = data[-1]

            y_pred = model(*x_batch)  #feed data into model          
            loss = loss_fn(y_pred, y_batch)
            
            #calculate error and adjust model params

            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
            avg_loss += loss.item() / len(train_loader) #gets the loss per epoch
        
        
        model.eval() #set model to eval mode for test data
        test_preds = np.zeros((len(test), output_dim))
    
        for i, x_batch in enumerate(test_loader):
            y_pred = sigmoid(model(*x_batch).detach().cpu().numpy()) #feed data into model

            test_preds[i * batch_size:(i+1) * batch_size, :] = y_pred #get test predictions
        
        #test_preds has the predictions for the entire test set now
        all_test_preds.append(test_preds) #append predictions to the record of all past predictions
        elapsed_time = time.time() - start_time
        print('Epoch {}/{} \t loss={:.4f} \t time={:.2f}s'.format(
              epoch + 1, n_epochs, avg_loss, elapsed_time))

    return model

def predict(model, test, output_dim, batch_size=512):
    
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
    
    model.eval() #set model to eval mode for test data
    test_preds = np.zeros((len(test), output_dim))
    
    for i, x_batch in enumerate(test_loader):
        y_pred = sigmoid(model(*x_batch).detach().cpu().numpy()) #feed data into model
        test_preds[i * batch_size:(i+1) * batch_size, :] = y_pred #get test predictions
        
    return test_preds

In [10]:
all_val_preds = []
all_test_preds = []
num_splits = 5
NUM_MODELS = 5

#Add in K fold 
random_state = 2019

#K fold splits
splits = list(StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=random_state).split(x_train,y_train))

#final validation predictions
final_val_preds = np.zeros((x_train.shape[0]))

#final test predictions to be stored in this var
final_test_preds = np.zeros((x_test.shape[0]))

start_time = time.time()
for fold in range(num_splits):
    tr_ind, val_ind = splits[fold]
    all_val_preds = []
    all_test_preds = []
    #print('Training set size: ', len(tr_ind))
    #print('Val set size: ', len(val_ind))
    x_training = x_train.iloc[tr_ind]
    y_training = y_train[tr_ind]
    #y_aux_training = y_aux_train[tr_ind]
    
    x_val = x_train.iloc[val_ind]
    y_val = y_train[val_ind]
    #y_aux_val = y_aux_train[val_ind]
    
    x_train_torch = torch.tensor(x_training.values, dtype=torch.float32).cuda()
    x_val_torch = torch.tensor(x_val.values, dtype=torch.float32).cuda()
    #y_train_torch = torch.tensor(np.hstack([y_training[:, np.newaxis], y_aux_training]), dtype=torch.float32).cuda()
    y_train_torch = torch.tensor(y_training, dtype=torch.float32).cuda()
    
    x_test_torch = torch.tensor(x_test.values, dtype=torch.float32).cuda()
    
    print(y_train_torch.shape)
    #print(y_train_torch.shape)
    
    #train_dataset = data.TensorDataset(x_train_torch, y_train_torch
    train_dataset = data.TensorDataset(x_train_torch, y_train_torch)
    val_dataset = data.TensorDataset(x_val_torch)
    test_dataset = data.TensorDataset(x_test_torch)

    for model_idx in range(NUM_MODELS):
        print('Model ', model_idx)
        seed_everything(1234 + model_idx)

        model = NeuralNet(input_size=x_training.shape[1], hidden_size=20, num_classes=2)
        try:
            model.cuda()
            #training using training and validation set
            model = train_model(model, train_dataset, val_dataset, output_dim=y_train_torch.shape[-1], 
                                 loss_fn=nn.BCEWithLogitsLoss(reduction='mean'))
        except:
            model.cuda()
            #training using training and validation set
            model = train_model(model, train_dataset, val_dataset, output_dim=y_train_torch.shape[-1], 
                                 loss_fn=nn.BCEWithLogitsLoss(reduction='mean'))
        
        #prediction on validation set (used for score measurement)
        val_pred = predict(model, val_dataset, output_dim=y_train_torch.shape[-1]) #val preds on the val split
        all_val_preds.append(val_pred)
        #print(len(val_pred))
        
        #prediction on entire test set (actual predictions to be submitted)
        test_pred = predict(model, test_dataset, output_dim=y_train_torch.shape[-1])
        all_test_preds.append(test_pred)
        
        print()
        
    #average validation prediction amongst all models
    avg_val = np.mean(all_val_preds, axis=0)[:, 0] #will be printed out per split
    final_val_preds[val_ind] += avg_val
    
    avg_test = np.mean(all_test_preds, axis=0)[:, 0]
    
    final_test_preds += avg_test

    y_true = y_train[val_ind] #true scores for this validation set
    y_identity = y_train_identity[val_ind] #true scores for the identity groups for this validation set
    #print(y_true)
    #print(y_identity)
    evaluator = JigsawEvaluator(y_true, y_identity)
    #print(avg_val)
    auc_score = evaluator.get_final_metric(avg_val)

    roc_score = roc_auc_score(y_train[val_ind], avg_val)
    print('Kaggle Score: ', auc_score)
    print('ROC score: ', roc_score)
    
    del x_train_torch
    del x_val_torch
    del y_train_torch
    del x_test_torch
    del train_dataset
    del val_dataset
    del test_dataset
    gc.collect()
    torch.cuda.empty_cache()
    
    print('=============End-of-Fold================')
    
end_time = time.time()
print('Time: ', end_time - start_time)

#Final combined score
y_true = y_train
y_identity = y_train_identity
evaluator = JigsawEvaluator(y_true, y_identity)
auc_score = evaluator.get_final_metric(final_val_preds)
print('Final Kaggle Score: ', auc_score)
print('Final ROC score: ', roc_auc_score(y_train, final_val_preds))

#average test predictions AGAIN this time by number of splits
final_test_preds /= num_splits
#print(final_test_preds)

torch.Size([1443899])
Model  0


ValueError: Target size (torch.Size([512])) must be the same as input size (torch.Size([512, 2]))

In [ ]:
x_training.values